In [1]:
import plotly.express as px
from utz import *
from ctbk import *
from geopy.distance import distance

import s3fs
import fsspec

## Create `ymrgtb_cd.json`

In [2]:
prv = read_parquet('s3://ctbk/aggregated/ymrgtb_cd_201306:202209.parquet')
prv

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
0,HB,2021,5,0,Customer,docked_bike,7583,12921780,2021-05-01
1,HB,2021,5,0,Subscriber,docked_bike,4204,3636441,2021-05-01
2,HB,2021,6,0,Customer,classic_bike,6112,7552060,2021-06-01
3,HB,2021,6,0,Customer,docked_bike,9348,14783466,2021-06-01
4,HB,2021,6,0,Subscriber,classic_bike,3829,2585386,2021-06-01
...,...,...,...,...,...,...,...,...,...
1047,NYC,2022,8,0,Customer,classic_bike,667347,929738595,2022-08-01
1048,NYC,2022,8,0,Customer,docked_bike,37247,79715610,2022-08-01
1049,NYC,2022,8,0,Customer,electric_bike,195581,259989518,2022-08-01
1050,NYC,2022,8,0,Subscriber,classic_bike,2206680,1754041394,2022-08-01


In [3]:
ymrgtb_cd = read_parquet('s3://ctbk/aggregated/ymrgtb_cd.parquet')
ymrgtb_cd

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
0,HB,2021,5,0,Customer,docked_bike,7583,12921780,2021-05-01
1,HB,2021,5,0,Subscriber,docked_bike,4204,3636441,2021-05-01
2,HB,2021,6,0,Customer,classic_bike,6112,7552060,2021-06-01
3,HB,2021,6,0,Customer,docked_bike,9348,14783466,2021-06-01
4,HB,2021,6,0,Subscriber,classic_bike,3829,2585386,2021-06-01
...,...,...,...,...,...,...,...,...,...
1062,NYC,2022,9,0,Customer,classic_bike,620418,830619866,2022-09-01
1063,NYC,2022,9,0,Customer,docked_bike,27979,59486661,2022-09-01
1064,NYC,2022,9,0,Customer,electric_bike,163679,214424477,2022-09-01
1065,NYC,2022,9,0,Subscriber,classic_bike,2185895,1714107735,2022-09-01


In [5]:
cols = [ 'Region', 'Start Year', 'Start Month', 'Gender', 'User Type', 'Rideable Type' ]

In [6]:
prv[cols].drop_duplicates(keep=False)

,Region,Start Year,Start Month,Gender,User Type,Rideable Type
0,HB,2021,5,0,Customer,docked_bike
1,HB,2021,5,0,Subscriber,docked_bike
2,HB,2021,6,0,Customer,classic_bike
3,HB,2021,6,0,Customer,docked_bike
4,HB,2021,6,0,Subscriber,classic_bike
...,...,...,...,...,...,...
1047,NYC,2022,8,0,Customer,classic_bike
1048,NYC,2022,8,0,Customer,docked_bike
1049,NYC,2022,8,0,Customer,electric_bike
1050,NYC,2022,8,0,Subscriber,classic_bike


In [7]:
ymrgtb_cd[cols].drop_duplicates(keep=False)

,Region,Start Year,Start Month,Gender,User Type,Rideable Type
0,HB,2021,5,0,Customer,docked_bike
1,HB,2021,5,0,Subscriber,docked_bike
2,HB,2021,6,0,Customer,classic_bike
3,HB,2021,6,0,Customer,docked_bike
4,HB,2021,6,0,Subscriber,classic_bike
...,...,...,...,...,...,...
1062,NYC,2022,9,0,Customer,classic_bike
1063,NYC,2022,9,0,Customer,docked_bike
1064,NYC,2022,9,0,Customer,electric_bike
1065,NYC,2022,9,0,Subscriber,classic_bike


In [23]:
ymrgtb_cd.value_counts()

Region  Start Year  Start Month  Gender  User Type   Rideable Type
HB      2021        5            0       Customer    docked_bike      1
NYC     2017        3            2       Subscriber  unknown          1
                    1            1       Subscriber  unknown          1
                                 2       Customer    unknown          1
                                         Subscriber  unknown          1
                                                                     ..
JC      2020        3            1       Customer    unknown          1
                                         Subscriber  unknown          1
                                 2       Customer    unknown          1
                                         Subscriber  unknown          1
NYC     2022        9            0       Subscriber  electric_bike    1
Length: 1067, dtype: int64

In [16]:
prv[prv[cols].duplicated(keep=False)]

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
4,NYC,2013,7,0,Customer,unknown,1,634,2013-07-01
0,NYC,2013,7,0,Customer,unknown,174889,259154198,2013-07-01
6,JC,2020,10,0,Customer,unknown,1957,3571604,2020-10-01
7,JC,2020,10,0,Subscriber,unknown,147,123338,2020-10-01
8,JC,2020,10,1,Customer,unknown,1474,2347706,2020-10-01
9,JC,2020,10,1,Subscriber,unknown,6538,4926177,2020-10-01
10,JC,2020,10,2,Customer,unknown,858,1784556,2020-10-01
11,JC,2020,10,2,Subscriber,unknown,2477,1947315,2020-10-01
0,JC,2020,10,0,Customer,unknown,3941,7078694,2020-10-01
1,JC,2020,10,0,Subscriber,unknown,322,245199,2020-10-01


In [24]:

pd.concat([ ymrgtb_cd[cols], prv2[cols] ]).drop_duplicates(keep=False)

,Region,Start Year,Start Month,Gender,User Type,Rideable Type
60,HB,2022,9,0,Customer,classic_bike
61,HB,2022,9,0,Customer,docked_bike
62,HB,2022,9,0,Customer,electric_bike
63,HB,2022,9,0,Subscriber,classic_bike
64,HB,2022,9,0,Subscriber,electric_bike
401,JC,2020,10,0,Customer,unknown
402,JC,2020,10,0,Subscriber,unknown
403,JC,2020,10,1,Customer,unknown
404,JC,2020,10,1,Subscriber,unknown
405,JC,2020,10,2,Customer,unknown


In [6]:
ymrgtb_cd[ymrgtb_cd.Month == to_dt('2022-08-01')]

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
55,HB,2022,8,0,Customer,classic_bike,15701,17275161,2022-08-01
56,HB,2022,8,0,Customer,docked_bike,571,1834462,2022-08-01
57,HB,2022,8,0,Customer,electric_bike,6756,5444782,2022-08-01
58,HB,2022,8,0,Subscriber,classic_bike,21798,12195645,2022-08-01
59,HB,2022,8,0,Subscriber,electric_bike,5442,2608320,2022-08-01
487,JC,2022,8,0,Customer,classic_bike,13760,17642913,2022-08-01
488,JC,2022,8,0,Customer,docked_bike,544,1947742,2022-08-01
489,JC,2022,8,0,Customer,electric_bike,5987,6163233,2022-08-01
490,JC,2022,8,0,Subscriber,classic_bike,35932,23505233,2022-08-01
491,JC,2022,8,0,Subscriber,electric_bike,8348,5113004,2022-08-01


In [5]:
ymrgtb_cd[ymrgtb_cd.Month == to_dt('2022-09-01')]

,Region,Start Year,Start Month,Gender,User Type,Rideable Type,Count,Duration,Month
60,HB,2022,9,0,Customer,classic_bike,13519,12994289,2022-09-01
61,HB,2022,9,0,Customer,docked_bike,432,1797347,2022-09-01
62,HB,2022,9,0,Customer,electric_bike,6536,5700767,2022-09-01
63,HB,2022,9,0,Subscriber,classic_bike,21404,12719812,2022-09-01
64,HB,2022,9,0,Subscriber,electric_bike,5479,2955768,2022-09-01
492,JC,2022,9,0,Customer,classic_bike,11252,13835630,2022-09-01
493,JC,2022,9,0,Customer,docked_bike,377,1421202,2022-09-01
494,JC,2022,9,0,Customer,electric_bike,4933,5318877,2022-09-01
495,JC,2022,9,0,Subscriber,classic_bike,32407,20519809,2022-09-01
496,JC,2022,9,0,Subscriber,electric_bike,7556,4672540,2022-09-01


In [14]:
(
    ymrgtb_cd
    .drop(columns='Month')
    .rename(columns={
        'Start Year': 'Year',
        'Start Month': 'Month',
    })
    .to_json('s3://ctbk/aggregated/ymrgtb_cd.json', 'records')
)

## Stations

In [4]:
stations = read_parquet('s3://ctbk/stations/ids.parquet')
stations

,Station Name,Latitude,Longitude
Station ID,,,
3040,Handlebar @ Gowanus,40.669800,-73.994980
3466,W 45 St & 6 Ave,40.756687,-73.982577
5190.09,Clinton St & Cherry St,40.711471,-73.986726
3645,Bike Mechanics at Riis Room B,40.672333,-73.997061
3477,39 St & 2 Ave - Citi Bike HQ at Industry City,40.655400,-74.010628
...,...,...,...
5769.04,Avenue C & E 18 St,40.730563,-73.973984
5763.03,Carmine St & 6 Ave,40.730386,-74.002150
5755.14,University Pl & E 8 St,40.731437,-73.994903


In [47]:
(
    stations
    .rename(columns={'Station Name': 'name', 'Latitude': 'lat', 'Longitude': 'lng' })
    .to_json('s3://ctbk/stations/ids.json', 'index')
)

In [46]:
os.stat('s3/ctbk/stations/ids.json').st_size

248842

## 202208 station counts

In [48]:
s_c_202208 = read_parquet('s3/ctbk/aggregated/s_c_202208.parquet')
s_c_202208

,Start Station ID,Count
0,2733.03,337
1,2782.02,457
2,2832.03,463
3,2872.02,89
4,2883.03,253
...,...,...
1660,JC105,2435
1661,JC106,2226
1662,JC107,62
1663,JC108,145


In [49]:
s_c_202208.rename(columns={'Start Station ID': 'id', 'Count': 'count'}).to_json('s3/ctbk/aggregated/s_c_202208.json', 'records')
os.stat('s3/ctbk/aggregated/s_c_202208.json').st_size

49105

In [50]:
s_c_202208.rename(columns={'Start Station ID': 'id', 'Count': 'count'}).to_json('s3://ctbk/aggregated/s_c_202208.json', 'records')

## All-time station counts

In [25]:
s_c = read_parquet('s3/ctbk/aggregated/s_c.parquet')
s_c

,Start Station ID,Count
0,116,309833
1,119,24998
2,120,64713
3,127,422203
4,128,420395
...,...,...
3240,JC108,1494
3241,JC109,3282
3242,JCSYS,4
3243,MTL-ECO51-1,2


In [27]:
s_c.rename(columns={'Start Station ID': 'id', 'Count': 'count'}).to_json('s3/ctbk/aggregated/s_c.json', 'records')
os.stat('s3/ctbk/aggregated/s_c.json').st_size

94428

In [2]:
se_c = read_parquet('s3://ctbk/aggregated/se_c.parquet').sort_values('Count')
se_c

,Start Station ID,End Station ID,Count
1904575,S 5th St & Kent Ave (Domino Park Movie Shoot),S 5th St & Kent Ave (Domino Park Movie Shoot),1
678302,3952,3531,1
678303,3952,3533,1
678305,3952,3536,1
678306,3952,3540,1
...,...,...,...
41075,237,150,92
1044649,500,228,92
935234,466,482,92
41086,237,174,92


In [3]:
se_c8 = read_parquet('s3/ctbk/aggregated/se_c_202208.parquet')
se_c8

,Start Station ID,End Station ID,Count
0,2733.03,2733.03,54
1,2733.03,2782.02,16
2,2733.03,2832.03,12
3,2733.03,2872.02,7
4,2733.03,2883.03,15
...,...,...,...
487261,JC109,JC105,5
487262,JC109,JC106,10
487263,JC109,JC107,2
487264,JC109,JC108,17


In [6]:
se_c8.to_json('s3/ctbk/aggregated/se_c_202208.json', 'records')

In [7]:
se_c8.to_parquet('s3/ctbk/aggregated/se_c_202208.parquet')

In [8]:
ymrse_c_202208 = read_parquet('s3/ctbk/aggregated/ymrse_c_202208.parquet')
ymrse_c_202208

,Region,Start Year,Start Month,Start Station ID,End Station ID,Count,Month
0,HB,2022,8,HB101,5297.02,1,2022-08-01
1,HB,2022,8,HB101,HB101,104,2022-08-01
2,HB,2022,8,HB101,HB102,247,2022-08-01
3,HB,2022,8,HB101,HB103,19,2022-08-01
4,HB,2022,8,HB101,HB105,41,2022-08-01
...,...,...,...,...,...,...,...
487261,NYC,2022,8,8841.03,8795.01,23,2022-08-01
487262,NYC,2022,8,8841.03,8795.03,7,2022-08-01
487263,NYC,2022,8,8841.03,8799.01,16,2022-08-01
487264,NYC,2022,8,8841.03,8811.01,16,2022-08-01


In [16]:
hc = (
    ymrse_c_202208
    [ymrse_c_202208.Region != 'NYC']
    .drop(columns=['Start Year', 'Start Month', 'Month', 'Region'])
    .rename(columns={
        'Start Station ID': 'start',
        'End Station ID': 'end',
        'Count': 'count',
    })
)
hc

,start,end,count
0,HB101,5297.02,1
1,HB101,HB101,104
2,HB101,HB102,247
3,HB101,HB103,19
4,HB101,HB105,41
...,...,...,...
5233,JC109,JC105,5
5234,JC109,JC106,10
5235,JC109,JC107,2
5236,JC109,JC108,17


In [17]:
hc.to_json('s3/ctbk/aggregated/ymrse_c_202208_hudco.json', 'records')
os.stat('s3/ctbk/aggregated/ymrse_c_202208_hudco.json').st_size

222694

In [18]:
hc.to_json('s3://ctbk/aggregated/ymrse_c_202208_hudco.json', 'records')

In [20]:
ymrs_c = read_parquet('s3/ctbk/aggregated/ymrs_c.parquet')
ymrs_c

,Region,Start Year,Start Month,Start Station ID,Count,Month
0,HB,2021,5,HB102,2281,2021-05-01
1,HB,2021,5,HB103,2051,2021-05-01
2,HB,2021,5,HB203,1148,2021-05-01
3,HB,2021,5,HB301,101,2021-05-01
4,HB,2021,5,HB302,308,2021-05-01
...,...,...,...,...,...,...
89911,NYC,2022,8,8795.01,568,2022-08-01
89912,NYC,2022,8,8795.03,233,2022-08-01
89913,NYC,2022,8,8799.01,246,2022-08-01
89914,NYC,2022,8,8811.01,163,2022-08-01


In [21]:
rs_c = read_parquet('s3/ctbk/aggregated/rs_c.parquet')
rs_c

,Region,Start Station ID,Count
0,HB,HB101,42167
1,HB,HB102,36718
2,HB,HB103,50877
3,HB,HB105,18839
4,HB,HB201,17901
...,...,...,...
3240,NYC,8795.01,5047
3241,NYC,8795.03,1688
3242,NYC,8799.01,2145
3243,NYC,8811.01,1371


In [24]:
rs_c.rename(columns={'Start Station ID': 'id', 'Count': 'count'}).drop(columns='Region').to_json('s3/ctbk/aggregated/rs_c.json', 'records')
os.stat('s3/ctbk/aggregated/rs_c.json').st_size

94383

In [43]:
s_c.rename(columns={'Start Station ID': 'id', 'Count': 'count'}).to_json('s3://ctbk/aggregated/s_c.json', 'records')

In [33]:
d1 = rs_c.drop(columns=['Region']).sort_values(['Start Station ID', 'Count']).reset_index(drop=True)
d2 = s_c.sort_values(['Start Station ID', 'Count']).reset_index(drop=True)

In [34]:
d1

,Start Station ID,Count
0,116,309833
1,119,24998
2,120,64713
3,127,422203
4,128,420395
...,...,...
3240,JC105,25772
3241,JC106,25832
3242,JC107,674
3243,JC108,1494


In [35]:
d2

,Start Station ID,Count
0,116,309833
1,119,24998
2,120,64713
3,127,422203
4,128,420395
...,...,...
3240,JC108,1494
3241,JC109,3282
3242,JCSYS,4
3243,MTL-ECO51-1,2


In [38]:
d1.Count.sum(), d2.Count.sum()

(161908923, 161908933)

In [39]:
s1 = set(d1['Start Station ID'].tolist())
s2 = set(d2['Start Station ID'].tolist())
s1.difference(s2)

set()

In [40]:
s2.difference(s1)

{'JCSYS', 'MTL-ECO51-1', 'S 5th St & Kent Ave (Domino Park Movie Shoot)'}

In [42]:
m = d1.merge(d2, on='Start Station ID')
m[m.Count_x != m.Count_y]

,Start Station ID,Count_x,Count_y
315,3197,34,2735
316,3197,2701,2735
338,3215,8,3645
339,3215,3637,3645
2185,5886.02,1,44177
2186,5886.02,44176,44177


In [36]:
(d1 == d2).all()

Start Station ID    False
Count               False
dtype: bool

In [2]:
%%time
sq = read_sql_table('agg', 'sqlite:///s3/ctbk/aggregated/se_c.db')#.sort_values('Count')
sq

CPU times: user 2.98 s, sys: 190 ms, total: 3.17 s
Wall time: 3.18 s


,Start Station ID,End Station ID,Count
0,116,116,73
1,116,119,1
2,116,120,9
3,116,127,73
4,116,128,70
...,...,...,...
1904571,JCSYS,JC005,1
1904572,JCSYS,JC078,1
1904573,JCSYS,JCSYS,1
1904574,MTL-ECO51-1,MTL-ECO51-1,1


In [4]:
sids = Series(concat([ sq['Start Station ID'], sq['End Station ID'] ]).unique(), name='ID').sort_values()
sids

0                                                 116
1                                                 119
2                                                 120
3                                                 127
4                                                 128
                            ...                      
3240                                            JC108
3241                                            JC109
3242                                            JCSYS
3243                                      MTL-ECO51-1
3244    S 5th St & Kent Ave (Domino Park Movie Shoot)
Name: ID, Length: 3257, dtype: object

In [5]:
idx2id = sids.to_frame().reset_index(drop=True)
idx2id.index.name = 'idx'
id2idx = idx2id.reset_index().set_index('ID')['idx']
idx2id = idx2id.ID
id2idx

ID
116                                                 0
119                                                 1
120                                                 2
127                                                 3
128                                                 4
                                                 ... 
JC108                                            3252
JC109                                            3253
JCSYS                                            3254
MTL-ECO51-1                                      3255
S 5th St & Kent Ave (Domino Park Movie Shoot)    3256
Name: idx, Length: 3257, dtype: int64

In [6]:
idx2id

idx
0                                                 116
1                                                 119
2                                                 120
3                                                 127
4                                                 128
                            ...                      
3252                                            JC108
3253                                            JC109
3254                                            JCSYS
3255                                      MTL-ECO51-1
3256    S 5th St & Kent Ave (Domino Park Movie Shoot)
Name: ID, Length: 3257, dtype: object

In [7]:
sq_id = (
    sq
    .merge(id2idx, left_on='Start Station ID', right_index=True)
    .rename(columns={'idx': 'start_idx'})
    .drop(columns='Start Station ID')
    .merge(id2idx, left_on='End Station ID', right_index=True)
    .rename(columns={'idx': 'end_idx'})
    .drop(columns='End Station ID')
)
sq_id

,Count,start_idx,end_idx
0,73,0,0
910,5,1,0
1547,4,2,0
2356,73,3,0
3322,72,4,0
...,...,...,...
1904365,5,3250,3251
1904424,13,3251,3251
1904493,11,3252,3251
1904567,2,3253,3251


In [9]:
%%time
sq_id.to_sql('agg_id', 'sqlite:///s3/ctbk/aggregated/se_c_id.db', index=False)

CPU times: user 6.05 s, sys: 653 ms, total: 6.7 s
Wall time: 11.6 s


1904576

In [10]:
len(sq['Start Station ID'].unique()), len(sq['End Station ID'].unique())

(3245, 3257)

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///s3/ctbk/aggregated/se_c_id.db', echo=False)

In [12]:
id2idx.to_sql('id2idx', engine)

3257

In [13]:
idx2id.to_sql('idx2id', engine)

3257

In [7]:
sql = 'select t.start_id, i.ID as end_id, t.Count from (select i.ID as start_id, a.end_idx, a.Count from agg_id a left join idx2id i on a.start_idx=i.idx) t left join idx2id i on t.end_idx=i.idx'
ss = pd.read_sql(sql, engine)
ss

,start_id,end_id,Count
0,116,116,73
1,119,116,5
2,120,116,4
3,127,116,73
4,128,116,72
...,...,...,...
1904571,JC106,JC107,5
1904572,JC107,JC107,13
1904573,JC108,JC107,11
1904574,JC109,JC107,2


In [15]:
se2 = (
    se
    .rename(columns={'Start Station ID': 'start_id', 'End Station ID': 'end_id'})
    .sort_values(['start_id', 'end_id', 'Count'])
    .reset_index(drop=True)
)
se2

,start_id,end_id,Count
0,116,116,73
1,116,119,1
2,116,120,9
3,116,127,73
4,116,128,70
...,...,...,...
1904571,JCSYS,JC005,1
1904572,JCSYS,JC078,1
1904573,JCSYS,JCSYS,1
1904574,MTL-ECO51-1,MTL-ECO51-1,1


In [14]:
ss2 = ss.sort_values(['start_id', 'end_id', 'Count']).reset_index(drop=True)
ss2

,start_id,end_id,Count
0,116,116,73
1,116,119,1
2,116,120,9
3,116,127,73
4,116,128,70
...,...,...,...
1904571,JCSYS,JC005,1
1904572,JCSYS,JC078,1
1904573,JCSYS,JCSYS,1
1904574,MTL-ECO51-1,MTL-ECO51-1,1


In [16]:
(se2 == ss2).all()

start_id    True
end_id      True
Count       True
dtype: bool

In [ ]:
engine

In [25]:
sids.reset_index()

,index,ID
0,0,116
1,1,119
2,2,120
3,3,127
4,4,128
...,...,...
3252,3240,JC108
3253,3241,JC109
3254,3242,JCSYS
3255,3243,MTL-ECO51-1


In [20]:
len(fss)

3257

In [ ]:
so

In [13]:
se[['Start Station ID', 'End Station ID']].drop_duplicates()

,Start Station ID,End Station ID
1904575,S 5th St & Kent Ave (Domino Park Movie Shoot),S 5th St & Kent Ave (Domino Park Movie Shoot)
678302,3952,3531
678303,3952,3533
678305,3952,3536
678306,3952,3540
...,...,...
41075,237,150
1044649,500,228
935234,466,482
41086,237,174


In [12]:
se.index

Int64Index([1904575,  678302,  678303,  678305,  678306,  678307,  678309,
             678310,  678311,  678312,
            ...
             935238,   24642,  935237, 1044639,   41076,   41075, 1044649,
             935234,   41086,  169712],
           dtype='int64', length=1904576)

In [9]:
ses = se.sort_values(['Start Station ID', 'End Station ID']).reset_index(drop=True)
ses

,Start Station ID,End Station ID,Count
0,116,116,73
1,116,119,1
2,116,120,9
3,116,127,73
4,116,128,70
...,...,...,...
1904571,JCSYS,JC005,1
1904572,JCSYS,JC078,1
1904573,JCSYS,JCSYS,1
1904574,MTL-ECO51-1,MTL-ECO51-1,1


In [11]:
ses.to_parquet('s3/ctbk/aggregated/se_c3.parquet')

In [10]:
ses.index

RangeIndex(start=0, stop=1904576, step=1)

In [8]:
len(ymrse1) - len(se)

17218792

In [13]:
se[['Start Station ID', 'End Station ID']].drop_duplicates()

,Start Station ID,End Station ID
1904575,S 5th St & Kent Ave (Domino Park Movie Shoot),S 5th St & Kent Ave (Domino Park Movie Shoot)
678302,3952,3531
678303,3952,3533
678305,3952,3536
678306,3952,3540
...,...,...
41075,237,150
1044649,500,228
935234,466,482
41086,237,174


In [2]:
ymrse1 = read_parquet('s3/ctbk/aggregated/ymrse_c.parquet.bak').sort_values('Count')

In [3]:
y1 = ymrse1.drop(columns=['Count', 'Month'])

rh = y1.groupby(y1.columns.tolist()).size().sort_values().rename('dupes').reset_index()
rh[rh.dupes > 1]

,Region,Start Year,Start Month,Start Station ID,End Station ID,dupes
19119906,JC,2020,10,3276,3205,2
19119907,JC,2020,10,3278,3281,2
19119908,JC,2020,10,3276,3206,2
19119909,JC,2020,10,3278,3280,2
19119910,JC,2020,10,3275,3639,2
...,...,...,...,...,...,...
19121632,JC,2020,10,3206,3679,2
19121633,JC,2020,10,3206,3640,2
19121634,JC,2020,10,3206,3639,2
19121635,JC,2020,10,3207,3202,2


In [4]:
ih = y1.index.to_series().sort_values().value_counts()
ih

0         111
24449     111
24443     111
24444     111
24445     111
         ... 
477583      1
477584      1
477585      1
477586      1
487265      1
Length: 487266, dtype: int64

In [2]:
ymrse2 = read_parquet('s3/ctbk/aggregated/ymrse_c.parquet').sort_values('Count')
#ymrse = ss.merge(stations['Station Name'].rename('Start Station Name'), left_on='Start Station ID', right_index=True, how='left')
#ymrse = ss.merge(stations['Station Name'].rename('End Station Name'), left_on='End Station ID', right_index=True, how='left')

In [8]:
ymrse1

,Region,Start Year,Start Month,Start Station ID,End Station ID,Count,Month
50501,NYC,2020,11,3118,3646,1,2020-11-01
28667,NYC,2017,1,3137,3173,1,2017-01-01
246638,NYC,2021,11,6998.08,5989.02,1,2021-11-01
108182,NYC,2018,10,3529,3390,1,2018-10-01
28663,NYC,2017,1,3137,3169,1,2017-01-01
...,...,...,...,...,...,...,...
451580,NYC,2022,7,8141.01,8141.01,2159,2022-07-01
303217,NYC,2021,6,6912.01,6912.01,2177,2021-06-01
6206,NYC,2015,5,2006,2006,3056,2015-05-01
6361,NYC,2015,7,2006,2006,3216,2015-07-01


In [3]:
dupes = ymrse1.drop(columns=['Count', 'Month']).duplicated(keep=False)
drs = ymrse1[dupes]
drs

,Region,Start Year,Start Month,Start Station ID,End Station ID,Count,Month
78906,NYC,2013,7,164,504,1,2013-07-01
14,JC,2020,10,3184,3206,1,2020-10-01
50,JC,2020,10,3185,3191,1,2020-10-01
80,JC,2020,10,3185,3280,1,2020-10-01
86,JC,2020,10,3185,3678,1,2020-10-01
...,...,...,...,...,...,...,...
299,JC,2020,10,3194,3195,216,2020-10-01
2400,JC,2020,10,3192,3192,228,2020-10-01
663,JC,2020,10,3205,3205,275,2020-10-01
206,JC,2020,10,3192,3192,426,2020-10-01


In [4]:
drs.Region.value_counts()

JC     3460
NYC       2
Name: Region, dtype: int64

In [5]:
drs[['Start Year', 'Start Month']].value_counts()

Start Year  Start Month
2020        10             3460
2013        7                 2
dtype: int64

In [6]:
ssh = drs[['Start Station ID', 'End Station ID']].value_counts().sort_values()
ssh

Start Station ID  End Station ID
164               504               2
3184              3186              2
                  3187              2
                  3192              2
                  3194              2
                                   ..
3792              3792              2
                  3209              2
3791              3270              2
3792              3278              2
                  4069              2
Length: 1731, dtype: int64

In [3]:
# [['Region', 'Start Year', 'Start Month', 'Start Station'
ymrse1.drop(columns=['Count', 'Month']).drop_duplicates()

,Region,Start Year,Start Month,Start Station ID,End Station ID
50501,NYC,2020,11,3118,3646
28667,NYC,2017,1,3137,3173
246638,NYC,2021,11,6998.08,5989.02
108182,NYC,2018,10,3529,3390
28663,NYC,2017,1,3137,3169
...,...,...,...,...,...
451580,NYC,2022,7,8141.01,8141.01
303217,NYC,2021,6,6912.01,6912.01
6206,NYC,2015,5,2006,2006
6361,NYC,2015,7,2006,2006


In [ ]:
ymrse1.to_parquet()

In [6]:
ymrse2.index.to_series().sort_values()

0                  0
1                  1
2                  2
3                  3
4                  4
              ...   
19121632    19121632
19121633    19121633
19121634    19121634
19121635    19121635
19121636    19121636
Length: 19121637, dtype: int64

In [3]:
ymrse2

,Region,Start Year,Start Month,Start Station ID,End Station ID,Count,Month
0,HB,2021,5,HB102,4947.04,1,2021-05-01
12747761,NYC,2021,3,6611.02,6297.07,1,2021-03-01
12747760,NYC,2021,3,6611.02,6297.02,1,2021-03-01
12747759,NYC,2021,3,6611.02,6289.06,1,2021-03-01
12747758,NYC,2021,3,6611.02,6280.12,1,2021-03-01
...,...,...,...,...,...,...,...
111959,JC,2020,10,3206,3679,2,2020-10-01
111957,JC,2020,10,3206,3640,2,2020-10-01
111956,JC,2020,10,3206,3639,2,2020-10-01
111974,JC,2020,10,3207,3202,2,2020-10-01


In [4]:
ymrse2.drop(columns=['Count', 'Month']).drop_duplicates()

,Region,Start Year,Start Month,Start Station ID,End Station ID
0,HB,2021,5,HB102,4947.04
12747761,NYC,2021,3,6611.02,6297.07
12747760,NYC,2021,3,6611.02,6297.02
12747759,NYC,2021,3,6611.02,6289.06
12747758,NYC,2021,3,6611.02,6280.12
...,...,...,...,...,...
111959,JC,2020,10,3206,3679
111957,JC,2020,10,3206,3640
111956,JC,2020,10,3206,3639
111974,JC,2020,10,3207,3202


In [11]:
ymrse1.columns == ymrse2.columns

array([ True,  True,  True,  True,  True,  True,  True])

In [10]:
ij = ymrse1.merge(ymrse2, on=ymrse1.columns, how='inner')
ij

KeyError: Index(['Region', 'Start Year', 'Start Month', 'Start Station ID',
       'End Station ID', 'Count', 'Month'],
      dtype='object')

In [11]:
sss = (
    ss
    .groupby(['Start Station ID', 'Start Station Name'])
    ['Count']
    .sum()
    .sort_values()
    .reset_index()
    .rename(columns={
        'Start Station ID': 'Station ID',
        'Start Station Name': 'Station Name',
        'Count': 'Starts',
    })
    .set_index('Station ID')
)
sss

,Station Name,Starts
Station ID,,
3633,333 Johnson TEST 1,1
3385,2 Ave & E 105 St,1
3000,MLSWKiosk,2
3446,NYCBS Depot - STY - Valet Scan,2
3248,LPI Facility,2
...,...,...
402,Broadway & E 22 St,674547
426,West St & Chambers St,694014
435,W 21 St & 6 Ave,732278


In [13]:
sss = (
    ss
    .groupby('Start Station ID')
    ['Count']
    .sum()
    .sort_values()
    .reset_index()
    .rename(columns={
        'Start Station ID': 'Station ID',
        'Count': 'Starts',
    })
    .set_index('Station ID')
)
sss

,Starts
Station ID,
3633,1
3385,1
3000,2
3446,2
3248,2
...,...
402,674547
426,694014
435,732278


In [15]:
starts = sss.merge(stations, left_index=True, right_index=True)
starts

,Starts,Station Name,Latitude,Longitude
Station ID,,,,
3633,1,333 Johnson TEST 1,0.000000,0.000000
3385,1,2 Ave & E 105 St,40.789817,-73.942961
3000,2,MLSWKiosk,40.755467,-73.986536
3446,2,NYCBS Depot - STY - Valet Scan,0.000000,0.000000
3248,2,LPI Facility,0.000000,0.000000
...,...,...,...,...
402,674547,Broadway & E 22 St,40.740343,-73.989551
426,694014,West St & Chambers St,40.717548,-74.013221
435,732278,W 21 St & 6 Ave,40.741740,-73.994156


In [9]:
sss.reset_index()

,Start Station ID,Start Station Name,Count
0,3633,333 Johnson TEST 1,1
1,3385,2 Ave & E 105 St,1
2,3000,MLSWKiosk,2
3,3446,NYCBS Depot - STY - Valet Scan,2
4,3248,LPI Facility,2
...,...,...,...
3237,402,Broadway & E 22 St,674547
3238,426,West St & Chambers St,694014
3239,435,W 21 St & 6 Ave,732278
3240,497,E 17 St & Broadway,759344


In [5]:
id2name = ss.set_index('Start Station ID')['Start Station Name']

In [7]:
id2name

Start Station ID
3118       McGuinness Blvd & Eagle St
3137                  5 Ave & E 73 St
6998.08            E 72 St & Park Ave
3529             Lenox Ave & W 130 St
3137                  5 Ave & E 73 St
                      ...            
8141.01          Grant Ave & E 169 St
6912.01    7 Ave & Central Park South
2006           Central Park S & 6 Ave
2006           Central Park S & 6 Ave
2006           Central Park S & 6 Ave
Name: Start Station Name, Length: 19123368, dtype: object

In [6]:
scs = sxs(sss, id2name)

/usr/local/lib/python3.9/site-packages/utz/pnds.py:44: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  return concat(dfs, axis=1, **kwargs)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
scs

In [43]:
ss

,Region,Start Year,Start Month,Start Station ID,End Station ID,Count,Month,Start Station Name,End Station Name
50501,NYC,2020,11,3118,3646,1,2020-11-01,McGuinness Blvd & Eagle St,35 Ave & 10 St
28667,NYC,2017,1,3137,3173,1,2017-01-01,5 Ave & E 73 St,Riverside Blvd & W 67 St
246638,NYC,2021,11,6998.08,5989.02,1,2021-11-01,E 72 St & Park Ave,W 15 St & 6 Ave
108182,NYC,2018,10,3529,3390,1,2018-10-01,Lenox Ave & W 130 St,E 109 St & 3 Ave
28663,NYC,2017,1,3137,3169,1,2017-01-01,5 Ave & E 73 St,Riverside Dr & W 82 St
...,...,...,...,...,...,...,...,...,...
451580,NYC,2022,7,8141.01,8141.01,2159,2022-07-01,Grant Ave & E 169 St,Grant Ave & E 169 St
303217,NYC,2021,6,6912.01,6912.01,2177,2021-06-01,7 Ave & Central Park South,7 Ave & Central Park South
6206,NYC,2015,5,2006,2006,3056,2015-05-01,Central Park S & 6 Ave,Central Park S & 6 Ave
6361,NYC,2015,7,2006,2006,3216,2015-07-01,Central Park S & 6 Ave,Central Park S & 6 Ave


In [5]:
df = ss.reset_index(drop=True).drop(columns=['Start Year', 'Start Month', 'Start Station ID', 'End Station ID'])
df

,Region,Count,Month,Start Station Name,End Station Name
0,NYC,1,2020-11-01,McGuinness Blvd & Eagle St,35 Ave & 10 St
1,NYC,1,2017-01-01,5 Ave & E 73 St,Riverside Blvd & W 67 St
2,NYC,1,2021-11-01,E 72 St & Park Ave,W 15 St & 6 Ave
3,NYC,1,2018-10-01,Lenox Ave & W 130 St,E 109 St & 3 Ave
4,NYC,1,2017-01-01,5 Ave & E 73 St,Riverside Dr & W 82 St
...,...,...,...,...,...
19123363,NYC,2159,2022-07-01,Grant Ave & E 169 St,Grant Ave & E 169 St
19123364,NYC,2177,2021-06-01,7 Ave & Central Park South,7 Ave & Central Park South
19123365,NYC,3056,2015-05-01,Central Park S & 6 Ave,Central Park S & 6 Ave
19123366,NYC,3216,2015-07-01,Central Park S & 6 Ave,Central Park S & 6 Ave


In [6]:
sames = df[df['Start Station Name'] == df['End Station Name']]
sames

,Region,Count,Month,Start Station Name,End Station Name
129,NYC,1,2020-01-01,21 St & 38 Ave,21 St & 38 Ave
2213,NYC,1,2020-01-01,28 St & 36 Ave,28 St & 36 Ave
14360,NYC,1,2020-01-01,37 St & 24 Ave,37 St & 24 Ave
18870,NYC,1,2020-01-01,Adam Clayton Powell Blvd & W 115 St,Adam Clayton Powell Blvd & W 115 St
19349,NYC,1,2017-01-01,Willoughby Ave & Tompkins Ave,Willoughby Ave & Tompkins Ave
...,...,...,...,...,...
19123363,NYC,2159,2022-07-01,Grant Ave & E 169 St,Grant Ave & E 169 St
19123364,NYC,2177,2021-06-01,7 Ave & Central Park South,7 Ave & Central Park South
19123365,NYC,3056,2015-05-01,Central Park S & 6 Ave,Central Park S & 6 Ave
19123366,NYC,3216,2015-07-01,Central Park S & 6 Ave,Central Park S & 6 Ave


In [9]:
sames_hist = sames.groupby(sames.Month)['Count'].sum()
sames_hist

Month
2013-06-01     23624
2013-07-01     28092
2013-08-01     31484
2013-09-01     26560
2013-10-01     21911
               ...  
2022-04-01    130904
2022-05-01    178595
2022-06-01    212156
2022-07-01    232214
2022-08-01    236933
Name: Count, Length: 111, dtype: int64

In [10]:
all_hist = df.groupby(df.Month)['Count'].sum()
all_hist

Month
2013-06-01     559643
2013-07-01     843417
2013-08-01    1001958
2013-09-01    1034359
2013-10-01    1037712
               ...   
2022-04-01    2379428
2022-05-01    3016642
2022-06-01    3529207
2022-07-01    3595881
2022-08-01    3785470
Name: Count, Length: 111, dtype: int64

In [11]:
sames_hist / all_hist

Month
2013-06-01    0.042213
2013-07-01    0.033307
2013-08-01    0.031422
2013-09-01    0.025678
2013-10-01    0.021115
                ...   
2022-04-01    0.055015
2022-05-01    0.059203
2022-06-01    0.060114
2022-07-01    0.064578
2022-08-01    0.062590
Name: Count, Length: 111, dtype: float64

In [22]:
ss.groupby('Rideable Type')['Count'].sum().sort_values()

Rideable Type
docked_bike        204625
electric_bike     5043399
classic_bike     15334367
Name: Count, dtype: int64

In [17]:
d.dtypes

Ride ID                            object
Rideable Type                      object
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station Name                 object
Start Station ID                   object
End Station Name                   object
End Station ID                     object
Start Station Latitude            float64
Start Station Longitude           float64
End Station Latitude              float64
End Station Longitude             float64
Gender                              int64
User Type                          object
Start Region                       object
End Region                         object
dtype: object

In [35]:
url = 's3/ctbk/stations/llname_hists'
#url = 's3://ctbk/stations/llname_hists'
fs = fsspec.filesystem('file')
fs

In [52]:
all_url = f'{url}/all.parquet'
stations = read_parquet(all_url)
stations

,Station ID,Station Name,Latitude,Longitude,count
751328,6319.07,45 Ave & 21 St,40.747386,-73.947899,1
997930,6854.05,W 50 St & 9 Ave,40.763668,-73.989233,1
997929,6854.05,W 50 St & 9 Ave,40.763668,-73.989196,1
997928,6854.05,W 50 St & 9 Ave,40.763668,-73.989341,1
997927,6854.05,W 50 St & 9 Ave,40.763668,-73.989216,1
...,...,...,...,...,...
67046,402,Broadway & E 22 St,40.740343,-73.989551,1381677
109935,426,West St & Chambers St,40.717548,-74.013221,1422807
121011,435,W 21 St & 6 Ave,40.741740,-73.994156,1471744
258691,497,E 17 St & Broadway,40.737050,-73.990093,1552945


In [54]:
stations.set_index('Station ID').sort_index()

,Station Name,Latitude,Longitude,count
Station ID,,,,
116,W 17 St & 8 Ave,40.743000,-74.001000,2
116,W 17 St & 8 Ave,40.741776,-74.001497,622257
119,Park Ave & St Edwards St,40.696089,-73.978034,51147
120,Lexington Ave & Classon Ave,40.686768,-73.959282,126227
127,Barrow St & Hudson St,40.731724,-74.006744,848627
...,...,...,...,...
JC109,Bergen Ave & Sip Ave,40.731239,-74.064360,1
JCSYS,JCBS Depot,40.709651,-74.068601,2
JCSYS,JCBS Depot,40.709651,-74.068601,23


In [55]:
stns = read_parquet(f'{url}/modes.parquet')
stns

,Station Name,Latitude,Longitude
Station ID,,,
3040,Handlebar @ Gowanus,40.669800,-73.994980
3466,W 45 St & 6 Ave,40.756687,-73.982577
5190.09,Clinton St & Cherry St,40.711471,-73.986726
3645,Bike Mechanics at Riis Room B,40.672333,-73.997061
3477,39 St & 2 Ave - Citi Bike HQ at Industry City,40.655400,-74.010628
...,...,...,...
5769.04,Avenue C & E 18 St,40.730563,-73.973984
5763.03,Carmine St & 6 Ave,40.730386,-74.002150
5755.14,University Pl & E 8 St,40.731437,-73.994903


In [37]:
pqts = DF(fs.listdir(url))
pqts = pqts.name
pqts

0      /src/s3/ctbk/stations/llname_hists/201306.parquet
1      /src/s3/ctbk/stations/llname_hists/201307.parquet
2      /src/s3/ctbk/stations/llname_hists/201308.parquet
3      /src/s3/ctbk/stations/llname_hists/201309.parquet
4      /src/s3/ctbk/stations/llname_hists/201310.parquet
                             ...                        
106    /src/s3/ctbk/stations/llname_hists/202204.parquet
107    /src/s3/ctbk/stations/llname_hists/202205.parquet
108    /src/s3/ctbk/stations/llname_hists/202206.parquet
109    /src/s3/ctbk/stations/llname_hists/202207.parquet
110    /src/s3/ctbk/stations/llname_hists/202208.parquet
Name: name, Length: 111, dtype: object

In [39]:
%%time
dfs = [ read_parquet(pqt) for pqt in pqts.values ]
len(dfs)

CPU times: user 609 ms, sys: 214 ms, total: 823 ms
Wall time: 1.25 s


111

In [40]:
hists = concat(dfs)
hists

,Station ID,Station Name,Latitude,Longitude,count
0,116,W 17 St & 8 Ave,40.741776,-74.001497,4571
1,119,Park Ave & St Edwards St,40.696089,-73.978034,168
2,120,Lexington Ave & Classon Ave,40.686768,-73.959282,883
3,127,Barrow St & Hudson St,40.731724,-74.006744,5041
4,128,MacDougal St & Prince St,40.727103,-74.002971,4439
...,...,...,...,...,...
664983,JC109,Bergen Ave & Sip Ave,40.731448,-74.064507,1
664984,JC109,Bergen Ave & Sip Ave,40.731463,-74.064198,1
664985,JC109,Bergen Ave & Sip Ave,40.731476,-74.063674,1
664986,JC109,Bergen Ave & Sip Ave,40.731502,-74.063244,1


In [41]:
hists.groupby('Station ID')['count'].sum().sort_values()

Station ID
3439          1
3748          1
3471          1
3794          1
3442          1
         ...   
402     1381677
426     1422809
435     1471748
497     1552948
519     1856447
Name: count, Length: 3257, dtype: int64

In [42]:
hists.groupby(['Station ID', 'Station Name'])['count'].sum().sort_values()

Station ID  Station Name                   
5805.07     7 Ave & Bleecker St                      1
3471        NYCBS Depot - GOW Annex                  1
3442        Indiana                                  1
3439        Broadway & E 22 St - Valet Scan          1
3432        Forrest portable kiosk                   1
                                                ...   
402         Broadway & E 22 St                 1381677
426         West St & Chambers St              1422809
435         W 21 St & 6 Ave                    1471748
497         E 17 St & Broadway                 1552948
519         Pershing Square North              1652358
Name: count, Length: 3382, dtype: int64

In [43]:
reduced = hists.groupby(['Station ID', 'Station Name', 'Latitude', 'Longitude'])['count'].sum().reset_index().sort_values('count')
reduced

,Station ID,Station Name,Latitude,Longitude,count
751328,6319.07,45 Ave & 21 St,40.747386,-73.947899,1
997930,6854.05,W 50 St & 9 Ave,40.763668,-73.989233,1
997929,6854.05,W 50 St & 9 Ave,40.763668,-73.989196,1
997928,6854.05,W 50 St & 9 Ave,40.763668,-73.989341,1
997927,6854.05,W 50 St & 9 Ave,40.763668,-73.989216,1
...,...,...,...,...,...
67046,402,Broadway & E 22 St,40.740343,-73.989551,1381677
109935,426,West St & Chambers St,40.717548,-74.013221,1422807
121011,435,W 21 St & 6 Ave,40.741740,-73.994156,1471744
258691,497,E 17 St & Broadway,40.737050,-73.990093,1552945


In [44]:
def row_sketch(a):
    restsum = sum(a[1:])
    total = a[0] + restsum
    num = len(a)
    return {
        'mode_count': a[0],
        'second': a[1] if num > 1 else nan,
        'restsum': restsum,
        'total': total,
        'counts': a,
        'first/second': a[0] / a[1] if num > 1 else nan,
        'mode_pct': a[0] / total,
        'num': num,
    }

def mode_sketch(df, groupby, thresh=0.5, sum_key='count'):
    idx_name = df.index.name
    if not idx_name:
        raise RuntimeError('Index needs a name')
    if isinstance(groupby, str):
        groupby = [groupby]
    row_groups = df.reset_index().groupby([idx_name] + groupby)
    if sum_key in df:
        row_hist = row_groups[sum_key].sum()
    else:
        row_hist = row_groups.size().rename(sum_key)
    row_hist = row_hist.reset_index()
    counts = row_hist.groupby(idx_name)[sum_key].apply(lambda s: list(reversed(sorted(s.values))))
    row_sketches = counts.apply(row_sketch).apply(Series)
    below_thresh = row_sketches[row_sketches.mode_pct < thresh]
    if not below_thresh.empty:
        stderr.write(f'{len(below_thresh)} index entries with mode_pct < {thresh}:\n{below_thresh}\n')
    annotated = (
        row_hist
        .sort_values([idx_name, sum_key], ascending=False)
        .drop_duplicates(subset=idx_name)
        .set_index(idx_name)
    )
    annotated = sxs(annotated, row_sketches).drop(columns=[sum_key]).sort_values('mode_pct')
    return annotated

In [45]:
annotated_station_names = mode_sketch(reduced.set_index('Station ID')[['Station Name', 'count']], 'Station Name')
annotated_station_names

3 index entries with mode_pct < 0.5:
            mode_count   second  restsum   total                counts  \
Station ID                                                               
3040                49     44.0       80     129          [49, 44, 36]   
3466             50588  47708.0    54077  104665  [50588, 47708, 6369]   
5190.09          49232  47820.0    49273   98505  [49232, 47820, 1453]   

            first/second  mode_pct  num  
Station ID                               
3040            1.113636  0.379845    3  
3466            1.060367  0.483333    3  
5190.09         1.029527  0.499792    3  


,Station Name,mode_count,second,restsum,total,counts,first/second,mode_pct,num
Station ID,,,,,,,,,
3040,Handlebar @ Gowanus,49,44.0,80,129,"[49, 44, 36]",1.113636,0.379845,3
3466,W 45 St & 6 Ave,50588,47708.0,54077,104665,"[50588, 47708, 6369]",1.060367,0.483333,3
5190.09,Clinton St & Cherry St,49232,47820.0,49273,98505,"[49232, 47820, 1453]",1.029527,0.499792,3
3645,Bike Mechanics at Riis Room B,1,1.0,1,2,"[1, 1]",1.000000,0.500000,2
3477,39 St & 2 Ave - Citi Bike HQ at Industry City,14095,13964.0,13964,28059,"[14095, 13964]",1.009381,0.502334,2
...,...,...,...,...,...,...,...,...,...
5769.04,Avenue C & E 18 St,76156,NaN,0,76156,[76156],NaN,1.000000,1
5763.03,Carmine St & 6 Ave,287453,NaN,0,287453,[287453],NaN,1.000000,1
5755.14,University Pl & E 8 St,174178,NaN,0,174178,[174178],NaN,1.000000,1


In [46]:
annotated_stations = mode_sketch(reduced.set_index('Station ID')[['Latitude', 'Longitude', 'count',]], ['Latitude', 'Longitude',])
annotated_stations

8 index entries with mode_pct < 0.5:
            mode_count   second  restsum  total                counts  \
Station ID                                                              
4322.02           2659   2609.0     3339   5998     [2659, 2609, 730]   
4455.07           6568   6073.0     6917  13485     [6568, 6073, 844]   
4962.04           2517   2452.0     2962   5479     [2517, 2452, 510]   
5024.08           2459   2411.0     3136   5595     [2459, 2411, 725]   
5746.04          11534  11411.0    14138  25672  [11534, 11411, 2727]   
6551.03            190    142.0      264    454       [190, 142, 122]   
6617.02            358    294.0      482    840       [358, 294, 188]   
7311.10           6792   6705.0     8423  15215    [6792, 6705, 1718]   

            first/second  mode_pct  num  
Station ID                               
4322.02         1.019164  0.443314    3  
4455.07         1.081508  0.487060    3  
4962.04         1.026509  0.459390    3  
5024.08         1.0199

,Latitude,Longitude,mode_count,second,restsum,total,counts,first/second,mode_pct,num
Station ID,,,,,,,,,,
6551.03,40.754121,-73.980252,190,142.0,264,454,"[190, 142, 122]",1.338028,0.418502,3
6617.02,40.757632,-73.969306,358,294.0,482,840,"[358, 294, 188]",1.217687,0.426190,3
5024.08,40.705309,-74.006125,2459,2411.0,3136,5595,"[2459, 2411, 725]",1.019909,0.439500,3
4322.02,40.683125,-73.978951,2659,2609.0,3339,5998,"[2659, 2609, 730]",1.019164,0.443314,3
7311.10,40.782536,-73.979085,6792,6705.0,8423,15215,"[6792, 6705, 1718]",1.012975,0.446402,3
...,...,...,...,...,...,...,...,...,...,...
3618,40.748500,-73.941275,54936,NaN,0,54936,[54936],NaN,1.000000,1
3619,40.771361,-73.924615,44794,NaN,0,44794,[44794],NaN,1.000000,1
4090,40.823429,-73.924689,1284,NaN,0,1284,[1284],NaN,1.000000,1


In [47]:
hists[hists['Station ID'] == '6551.03']

,Station ID,Station Name,Latitude,Longitude,count
1457,6551.03,E 43 St & 5 Ave,40.754120,-73.980251,122
1458,6551.03,E 43 St & 5 Ave,40.754121,-73.980252,142
1459,6551.03,E 43 St & 5 Ave,40.754121,-73.980252,190


In [48]:
hists[hists['Station ID'] == '6551.03'].groupby(['Latitude', 'Longitude'])['count'].sum()

Latitude   Longitude 
40.754120  -73.980251    122
40.754121  -73.980252    142
           -73.980252    190
Name: count, dtype: int64

In [19]:
hists.sort_values('count')

,Station ID,Station Name,Latitude,Longitude,count
349185,7511.19,5 Ave & E 103 St,40.792001,-73.952754,1
160880,5303.08,Canal St & Rutgers St,40.714289,-73.990028,1
160879,5303.08,Canal St & Rutgers St,40.714289,-73.990000,1
160878,5303.08,Canal St & Rutgers St,40.714289,-73.990055,1
160877,5303.08,Canal St & Rutgers St,40.714289,-73.989976,1
...,...,...,...,...,...
35202,5329.03,West St & Chambers St,40.717548,-74.013221,27163
114965,6140.05,W 21 St & 6 Ave,40.741740,-73.994156,27700
108227,5329.03,West St & Chambers St,40.717548,-74.013221,30139
163798,5329.03,West St & Chambers St,40.717548,-74.013221,31185
